# building a stock program

- STEP 1
    - monitor 3 or 4 diverse stocks (yfinance? finnhub?)
    - PCAR, TSLA, AMZN, CRUDE OIL
    - read in stock data for these
- STEP 2
    - pass their own data as endog and the others as exog into a walking sarima
- STEP 2.5
    - feature engeneering. What else can I add as exog to increase accuracy? Unemploymeent? Inflation? etc..
    - How should I incorpoporate things other than Close
- STEP 3
    - set up an account with a small amount of money and let this invest it freely
    - maybe in my schwab? maybe on investopedia?


- Thoughts
    - switch to day change
    - switch from forecasting to just classification sell/hold
    - look into stats forecast package
    - add a lasso for feature selection

In [56]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import statsmodels.api as sm
import pmdarima as pm
import warnings
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [57]:
def make_stock_data(stocks,feature):
    info_list=[]
    if feature == 'Close':
        # Close
        for stock in stocks:
            info = yf.Ticker(stock).history(period='max')[['Close']]
            info_list.append(info)
    elif feature == 'Change':
        # Day Change
        for stock in stocks:
            info = yf.Ticker(stock).history(period='max')[['Open','Close']]
            info = info.Open-info.Close
            info_list.append(info)
    else: print('give valid feature: Change or Close')

    combo = pd.concat(info_list,axis=1)
    combo.columns = stocks

    combo = combo.dropna()
    combo.index = pd.to_datetime(combo.index)
    return combo

In [58]:
# need to keep exog rows how they are, but instead of endog being todays stock price, it needs to be tomorows stock price
# how to do this?
    # shift stock of interest up?
    # row 1 will have row 2, which is its tomorow
    # last row will have na which makes sense because we dont know it yet
    # will mess up date column because date will be date we are making prediciton (yesterday)

In [59]:
# SOMEHOW THIS BROKE

def set_up_endog_exog(data,tt,stock):
        lis = [x for x in data.columns]
        exs = lis.copy()
        exs.remove(stock)
        size = int(len(data)*tt)

        # shift endog up
        this_data = data.copy()
        this_data[stock] = this_data[stock].shift(-1) # automatically adds the na for tomoorw
        ## now in the data, the value we are trying to predict is lagged by 1.
        ## this means on today we have all of todays info, and are tryinging to predict tomorow
        train, test = this_data[0:size],this_data[size:len(this_data)]
        train_endog = train[[stock]]
        test_endog = test[[stock]]
        test_endog_ = [x for x in test_endog.values]
        history_endog = [x for x in train_endog.values]

        exs = lis.copy()
        exs.remove(stock)
        train_exog = train[exs]
        test_exog = test[exs]
        test_exog_ = [x for x in test_exog.values]
        history_exog = [x for x in train_exog.values]

        actuals = data[stock]

        return history_endog,history_exog,test_exog_,test_endog_,actuals


In [60]:
# look into endog params within model

def walking_sarimax(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    order_ = params['order']
    seasonal_order_ = params['seasonal_order']
    trend_ = params['trend']
    for stock in data.columns:
        history_endog,history_exog,test_exog_,test_endog_,actuals= set_up_endog_exog(data,tt,stock)

        preds = [np.nan]*len(history_endog)
        for t in range(len(test_endog_)):
            model = sm.tsa.statespace.SARIMAX(endog = history_endog,exog = history_exog,
                order=order_,seasonal_order=seasonal_order_,trend=trend_,
                    time_varying_regression=False,mle_regression=True,initialization='approximate_diffuse',
                        hamilton_representation=False)

            model_fit = model.fit(disp=False)
            output = model_fit.forecast(exog = test_exog_[t])
            history_endog.append(test_endog_[t])
            history_exog.append(test_exog_[t])
            preds.append(output[0])
            
            if math.isnan(test_endog_[t]) == False:
                j_preds.append(output[0])
                acts.append(test_endog_[t])
        dates = np.append((data.index.min()-pd.DateOffset(days=1)),np.array(data.index.date))
        walking_preds = pd.DataFrame(zip(dates,actuals,preds),columns=['Date',f'{stock}_TODAY_ACTUAL',f'{stock}_TOMOROW_PRED'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

In [61]:
combo = make_stock_data(['aapl','pcar'],'Close').dropna()
out,p,a = walking_sarimax(combo.tail(100),tt,params)

,Date,aapl_TODAY_ACTUAL,aapl_TOMOROW_PRED,pcar_TODAY_ACTUAL,pcar_TOMOROW_PRED,aapl_Pred
0,2022-03-21,168.338501,NaN,88.804825,NaN,NaN
1,2022-03-22,169.724533,NaN,87.57972,NaN,NaN
2,2022-03-23,173.573532,NaN,88.595657,NaN,NaN
3,2022-03-24,174.221664,NaN,88.615578,NaN,NaN
4,2022-03-25,175.099167,NaN,88.944267,NaN,NaN
...,...,...,...,...,...,...
95,2022-08-05,164.869995,167.112076,91.589996,91.976877,165.978578
96,2022-08-08,164.919998,165.978578,91.370003,91.592641,169.733032
97,2022-08-09,169.240005,169.733032,93.25,93.417092,169.165163
98,2022-08-10,168.490005,169.165163,93.0,93.248267,172.466011


In [63]:
px.line(out.set_index('Date'))

In [64]:
def walking_sarima(data,tt,params):
    walking_preds_list=[]
    j_preds=[]
    acts=[]
    for stock in data.columns:
        X=data[stock].to_numpy()

        size = int(len(X)*tt)
        train, test = X[0:size],X[size:len(X)]
        #add an np.nan onto the end of train to represent the unknown tomorow
        test = np.append(test,np.nan)
                
        history = [x for x in train]

        preds = [np.nan]*len(train)

        order_ = params['order']
        seasonal_order_ = params['seasonal_order']
        trend_ = params['trend']
        for t in range(len(test)):
            model = sm.tsa.statespace.SARIMAX(history,order=order_,seasonal_order=seasonal_order_,trend=trend_)

            model_fit = model.fit(disp=False)
            output = model_fit.forecast()

            history.append(test[t])
            preds.append(output[0])
            
            if math.isnan(test[t]) == False:
                j_preds.append(output[0])
                acts.append(test[t])

        X = np.append(X,np.nan)
        dates = np.append(np.array(data.index.date),(data.index.max()+pd.DateOffset(days=1))) # add tomro
        walking_preds = pd.DataFrame(zip(dates,X,preds),columns=['Date',f'{stock}_TODAY_ACTUAL',f'{stock}_TOMOROW_PRED'])
        walking_preds_list.append(walking_preds)
    walking_preds_all = pd.concat(walking_preds_list,axis=1).T.drop_duplicates().T
    return walking_preds_all,j_preds,acts

auto = pm.auto_arima(combo.spx, X=combo.drop('spx',axis=1).to_numpy(), start_p=2, d=None, start_q=2, max_p=15, max_d=2, max_q=7, start_P=1, D=None, 
start_Q=1, max_P=5, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False, 
information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=True, 
n_jobs=1, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, 
seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, random=False, 
random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, scoring='mse', scoring_args=None, 
with_intercept='auto', sarimax_kwargs=None)

# find some way to test
- start with X dollars
- if alg predicts price goes up, hold
- if alg predicts price goes down, sell

In [65]:
def decide(output,stock,feature):
    # if next pred higher than current actual, hold, else, sell
    output = output[['Date',f'{stock}_TODAY_ACTUAL',f'{stock}_TOMOROW_PRED']].dropna()
    todo = []
    act = f'{stock}_TODAY_ACTUAL'
    pre = f'{stock}_TOMOROW_PRED'
    if feature == 'Close':
        for i in range(len(output)-1):
            if output.iloc[i][act] <= output.iloc[i+1][pre]: # if tomorow pred is bigger than todays actual
                todo.append('HOLD')
            else:
                todo.append('SELL')
                
    if feature == 'Change':
        for i in range(len(output)-1):
            if output.iloc[i+1][pre]>0: # if tomorow tomorrow predicted day change is positive, hold
                todo.append('HOLD')
            else:
                todo.append('SELL')
    todo.append('?')
    output['Todo'] = todo
    return output

In [66]:

def run_test(info,feature):
    compare_list=[]
    for stock in stock_name:
        x = 100
        x2 = x
        x3 = x
        x4=x
        plann=[]
        hold=[]
        perfect=[]
        naive = []
        plan = decide(info,stock,feature)

        for i in range(1,len(plan)-1):
            row = plan.iloc[i]
            next_row = plan.iloc[i+1]
            last_row = plan.iloc[i-1]
            
            ## this logic has to be updated for change option, only works with close
            change = ((next_row[1]-row[1])/row[1])
            # using system
            if row.Todo =='HOLD':
                x = x + (x*change)
            # just holding
            x2 = x2+(x2*change)
            # if it was perfect
            if change >0:
                x3 = x3+(x3*change)
            # if positive today positive tmo
            if row[1]>last_row[1]:
                x4 = x4 + x4*change
            
            plann.append(x)
            hold.append(x2)
            perfect.append(x3)
            naive.append(x4)
        
        print(f'{stock}:')
        print(f'    -Perfect: {x3}$')
        print(f'    -Plan: {x}$')
        print(f'    -Hold: {x2}$')
        print(f'    -Naive: {x4}$')

        plan_frame = pd.DataFrame(zip(plan.Date,plann,['plan']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
        hold_frame = pd.DataFrame(zip(plan.Date,hold,['hold']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
        perfect_frame = pd.DataFrame(zip(plan.Date,perfect,['perfect']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
        naive_frame = pd.DataFrame(zip(plan.Date,naive,['naive']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])

        compare= pd.concat([plan_frame,hold_frame,perfect_frame,naive_frame])
        compare_list.append(compare)
    all_compare = pd.concat(compare_list)

    all_compare['indicator'] = all_compare['type']+' '+all_compare['stock']
    plot = px.line(x=all_compare.date,y=all_compare.val,color=all_compare.indicator)
    all_compare = all_compare.drop('indicator',axis=1)

    return all_compare,plot




In [67]:

def beat_market(stock_name,length,tt,params,feature):

    combo = make_stock_data(stock_name,feature)

    out,p,a= walking_sarimax(combo.tail(length),tt,params)

    out_plot = px.line(out.set_index('Date'))

    print(f'sarimax rmse: {np.sqrt(mean_squared_error(p,a))}') #14.16 

    all_compare,compare_plot = run_test(out,feature)

    return out,out_plot,all_compare,compare_plot

    

In [68]:
stock_name = ['pcar','amzn','tsla','oil','aapl']
params = {'order':[5,0,0],
        'seasonal_order':[0,0,0,5],
        'trend':'ct'}
length = 300
tt = 0.8

feature = 'Close'

out,out_plot,all_compare,compare_plot = beat_market(stock_name,length,tt,params,feature)
#13 ish

KeyboardInterrupt: 

In [ ]:
out_plot